# 什么是 Git

Git 诞生于 Linux 开源社区（特别是 Linux 的缔造者 Linus Torvalds），他们希望开发出自己的版本控制系统，他们对新系统制定了若干目标：

* 速度
* 简单的设计
* 对非线性开发模式的强力支持（允许成千上万个并行开发的分支）
* 完全分布式
* 有能力高效管理类似 Linux 内核一样的超大规模项目（速度和数据量）

自 2005 年诞生以来，Git 日臻成熟完善，在高度易用的同时，仍然保留着最初设定的目标。它的速度飞快，极其适合管理大项目，有着令人难以置信的非线性分支管理系统。

```{note}
那么，简单说，Git 究竟是怎样的一个系统呢？请注意接下来的内容非常重要，若你理解了 Git 思想的基本工作原理，用起来就会知其所以然，游刃有余。尽管 Git 用起来与其它版本控制系统非常相似，但它在对信息的存储和认知方式上却有很大差异，理解这些差异将有助于避免使用中的困惑。
```

## 直接记录快照，而非差异比较

Git 与其它版本控制系统的主要差别在于 Git 对待数据的方式。其他大部分系统以文件变更列表的方式存储信息，这类系统将它们存储的信息看做是一组基本文件和每个文件随时间逐步累计的差异。

![](../images/deltas.png)

Git 不按以上方式对待或保存数据。反之，Git 更像是把数据当做小型文件系统的一系列快照。在 Git 中，每当你提交更新或保存项目状态时，它基本上就会`对当时的全部文件创建一个快照并保存这个快照的索引`。为了效率，如果文件没有修改，Git 不再重新存储该文件，而是只保留一个链接指向之前存储的文件。

![](../images/snapshots.png)

稍后，我们在讨论 Git 分支管理时，将探究这种方式对待数据所能获得的益处。

## 近乎所有操作都在本地执行

在 Git 中的绝大多数操作都只需要访问本地文件和资源，一般不需要来自网络上其它计算机的信息。如果你习惯了所有操作都有网络延时开销的的集中式版本控制系统，Git 在这方面会让你感到速度之神赐给了 Git 超凡的能量。因为你在本地磁盘上就有项目的完整历史，所以大部分操作看起来瞬间完成。

这也意味着你在离线或者没有 VPN 时（比如你在飞机上），几乎可以进行任何操作。

## Git 保存完整性

Git 中所有数据在存储前都会计算校验和，然后以校验和来引用。这意味着不可能在 Git 不知情时更改任何文件内容或目录内容。这个功能构建在 Git 底层，是构成 Git 哲学不可或缺的部分。

Git 用以计算校验和的机制叫做 SHA-1 散列。这是一个由 40 个十六进制字符组成的字符串，基于 Git 中文件的内容或目录结构计算出来。SHA-1 哈希看起来是这样：

```
24b9da6552252987aa493b52f8696cd6d3b00373
```

Git 中使用这种哈希值的情况很多，你将经常看到这种哈希值。

## Git 一般只添加数据

你执行的 Git 操作，几乎只往 Git 数据库中添加数据。你很难使用 Git 从数据库中删除数据，也就是说 Git 几乎不会执行任何可能导致文件不可回复的操作。同别的 VCS 一样，未提交更新时有可能丢失或弄乱修改的内容。但是一旦你提交快照到 Git 中，就难以再丢失数据，特别是如果你定期推送数据库到其它仓库的话。

## 三种状态

Git 有三种状态，你的文件可能会处于其中之一：已提交（committed）、已修改（modified）和已暂存（staged）。

* 已修改表示修改了文件，但还没保存到数据库中。
* 已暂存表示对一个已修改文件的当前版本做了标记，使之包含在下次提交的快照中。
* 已提交表示数据已经安全地保存在本地数据库中。

这会让我们的 Git 项目拥有：工作区、暂存区以及 Git 目录。

![](../images/areas.png)

工作区是对某个版本独立提取出来的内容。这些从 Git 仓库的压缩数据库中提取出来的文件，放在磁盘上供你使用或修改。

The staging area is a file that stores information about what will go into your next commit.

Git 仓库目录是 Git 用来保存项目的元数据和对象数据库的地方。这是 Git 中最重要的部分，从其它计算机克隆仓库时，复制的就是这里的数据。

基本的 Git 工作流程如下：

1. 在工作区中修改文件
2. 将你想要下次提交的更改选择性地暂存，这样只会将更改的部分添加到暂存区。
3. 提交更新，找到暂存区的文件，将快照永久性存储到 Git 目录。

If a particular version of a file is in the Git directory, it's considered `committed`. If it has been modified and was added to the staging area, it is `staged`. And if if was changed since it was checked out but not been staged, it it `modified`.